Install packages (optional if already used pip and requirements.txt)

In [1]:
# ! pip install datasets
# ! pip install accelerate
# ! pip install evaluate

Load packages

In [2]:
from transformers import (
    AutoTokenizer,
    DataCollatorWithPadding,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer
)
from datasets import Dataset
import evaluate
import numpy as np
import pandas as pd

/home/alex/git/practical-ml-seminar/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Define helper functions

In [3]:
tokenizer = AutoTokenizer.from_pretrained('distilbert/distilbert-base-uncased', model_max_length=512)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
def preprocess_function(examples):
    return tokenizer(examples['text'], truncation=True)


accuracy = evaluate.load('accuracy')
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

- Load dataset
- Calculate id2label and label2id dictionaries
- Label, shuffle, stratify, and split

In [4]:
df = pd.read_csv('data/GB-GOV-1.csv')
unique_labels = df.label.unique()
id2label = {i: label for i, label in enumerate(unique_labels)}
label2id = {id2label[i]: i for i in id2label.keys()}
dataset = Dataset.from_pandas(df).class_encode_column("label").train_test_split(
    test_size=0.3,
    stratify_by_column="label",
    shuffle=True,
)

Casting to class labels: 100%|██████████| 3878/3878 [00:00<00:00, 253831.32 examples/s]


Tokenize dataset

In [5]:
tokenized_data = dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 1164/1164 [00:00<00:00, 22385.93 examples/s]


Load and set up model

In [6]:
model = AutoModelForSequenceClassification.from_pretrained(
    'distilbert/distilbert-base-uncased', num_labels=len(id2label.keys()), id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert/distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Set up training arguments and trainer

In [7]:
training_args = TrainingArguments(
    output_dir='models/climate-classifier',
    learning_rate=1e-5, # This can be tweaked depending on how loss progresses
    per_device_train_batch_size=36, # These should be tweaked to match GPU VRAM
    per_device_eval_batch_size=36,
    num_train_epochs=10,
    weight_decay=0.01,
    evaluation_strategy='epoch',
    save_strategy='epoch',
    logging_strategy='epoch',
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['test'],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

Initialize training

In [8]:
trainer.train()

                                                
 10%|█         | 76/760 [02:43<21:11,  1.86s/it]Checkpoint destination directory models/climate-classifier/checkpoint-76 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 0.43150314688682556, 'eval_accuracy': 0.8213058419243986, 'eval_runtime': 21.0315, 'eval_samples_per_second': 55.345, 'eval_steps_per_second': 1.569, 'epoch': 1.0}


                                                  
 20%|██        | 152/760 [05:40<15:15,  1.51s/it]Checkpoint destination directory models/climate-classifier/checkpoint-152 already exists and is non-empty.Saving will proceed but saved results may be invalid.


{'eval_loss': 0.38561081886291504, 'eval_accuracy': 0.834192439862543, 'eval_runtime': 16.5845, 'eval_samples_per_second': 70.186, 'eval_steps_per_second': 1.99, 'epoch': 2.0}


                                                   
 30%|███       | 228/760 [08:31<14:30,  1.64s/it]Checkpoint destination directory models/climate-classifier/checkpoint-228 already exists and is non-empty.Saving will proceed but saved results may be invalid.


...

                                                 
100%|██████████| 760/760 [30:05<00:00,  1.75s/it]

{'eval_loss': 0.4387195110321045, 'eval_accuracy': 0.8470790378006873, 'eval_runtime': 18.77, 'eval_samples_per_second': 62.014, 'eval_steps_per_second': 1.758, 'epoch': 10.0}


100%|██████████| 760/760 [30:06<00:00,  2.38s/it]

{'train_runtime': 1806.9874, 'train_samples_per_second': 15.019, 'train_steps_per_second': 0.421, 'train_loss': 0.26499609696237664, 'epoch': 10.0}


TrainOutput(global_step=760, training_loss=0.26499609696237664, metrics={'train_runtime': 1806.9874, 'train_samples_per_second': 15.019, 'train_steps_per_second': 0.421, 'train_loss': 0.26499609696237664, 'epoch': 10.0})